In [2]:
import pyLDAvis
import pyLDAvis.sklearn

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
# from glob import glob

from bagel import DataSet, DataFile

# import matplotlib.pyplot as plt
# import numpy
# import seaborn
pyLDAvis.enable_notebook()

In [3]:
# run this at least once
# import nltk
# nltk.download('stopwords')

In [4]:
FILE_PATTERN = 'zero-carbon-bill/input/*.json'
TOKEN_PATTERN = r'\b[a-zA-ZāēīōūĀĒĪŌŪ]{3,}\b'

# Tweak this parameter until it brings joy
NUMBER_OF_TOPICS = 5

FILE_PATTERN = 'zero-carbon-bill/input/*.json'
# FILE_PATTERN = 'test-data/*.txt'

data_set = DataSet(FILE_PATTERN)


In [6]:
dataset = DataSet('zero-carbon-bill/input/*.json')
dataset.make_lda_model()
dataset.print_topics(num_topics=20, num_words=15)


0: 0.015*"emissions" + 0.012*"need" + 0.011*"nz" + 0.010*"climate" + 0.009*"newzealand" + 0.009*"change" + 0.009*"carbon" + 0.008*"gases" + 0.008*"people" + 0.007*"must"
99.72% of the data is in this topic

1: 0.015*"time" + 0.015*"government" + 0.014*"need" + 0.011*"carbon" + 0.010*"world" + 0.009*"must" + 0.009*"planet" + 0.008*"nz" + 0.008*"important" + 0.007*"new"
99.14% of the data is in this topic

2: 0.028*"climate" + 0.021*"change" + 0.019*"nz" + 0.013*"many" + 0.010*"biochar" + 0.010*"earth" + 0.010*"years" + 0.009*"sun" + 0.008*"bill" + 0.008*"massive"
14.02% of the data is in this topic

3: 0.019*"emissions" + 0.016*"cars" + 0.014*"would" + 0.013*"electric" + 0.010*"change" + 0.010*"temperature" + 0.009*"years" + 0.008*"even" + 0.008*"global" + 0.008*"net"
21.98% of the data is in this topic

4: 0.034*"emissions" + 0.027*"government" + 0.022*"carbon" + 0.016*"zero" + 0.015*"climate" + 0.014*"change" + 0.013*"newzealand" + 0.010*"need" + 0.010*"like" + 0.009*"bill"
99.59% of 

In [7]:
dataset.print_files()


99.72% in topic 0.	zero-carbon-bill/input/01290_Lindsay_Brown_Redacted.pdf.txt.json
99.14% in topic 16.	zero-carbon-bill/input/00596_Anonymous.pdf.txt.json
14.03% in topic 4.	zero-carbon-bill/input/01197_Chris_Rapson_Redacted.pdf.txt.json
22.01% in topic 0.	zero-carbon-bill/input/00142_Matthew_Weir_Redacted.pdf.txt.json
99.59% in topic 2.	zero-carbon-bill/input/00117_R_Ca_Redacted.pdf.txt.json
24.79% in topic 3.	zero-carbon-bill/input/09586_Peter_Francis_Kerr_Redacted.pdf.txt.json
99.41% in topic 12.	zero-carbon-bill/input/01202_Anthony_Weddle_Redacted.pdf.txt.json
79.70% in topic 4.	zero-carbon-bill/input/09203_Dinah_Sally_Hawken_Redacted.pdf.txt.json
98.44% in topic 19.	zero-carbon-bill/input/01193_Jennifer_Ward_Redacted.pdf.txt.json
98.14% in topic 18.	zero-carbon-bill/input/02637_Olivia_Elizabeth_Teryle_Yates_Redacted.pdf.txt.json
99.56% in topic 18.	zero-carbon-bill/input/09574_Sharyn_Joy_Watkins_Redacted.pdf.txt.json
99.65% in topic 12.	zero-carbon-bill/input/02523_Alison_Burt.pd

99.46% in topic 8.	zero-carbon-bill/input/01630_Alec_Messenger_Redacted.pdf.txt.json
99.18% in topic 1.	zero-carbon-bill/input/08408_Inger_Perkins_Redacted.pdf.txt.json
25.63% in topic 1.	zero-carbon-bill/input/04923_Anonymous.pdf.txt.json
98.14% in topic 19.	zero-carbon-bill/input/09481_Terry_Drayton_Redacted.pdf.txt.json
97.36% in topic 6.	zero-carbon-bill/input/08601_Keith_Morfett_Redacted.pdf.txt.json
94.06% in topic 0.	zero-carbon-bill/input/02449_Peter_Harley_Deacon_Redacted.pdf.txt.json
99.54% in topic 0.	zero-carbon-bill/input/00055_Thomas_Kulpe_Redacted.pdf.txt.json
98.56% in topic 7.	zero-carbon-bill/input/09478_Samantha_Hoskins_Redacted.pdf.txt.json
96.94% in topic 1.	zero-carbon-bill/input/09369_Graeme_Ball_Redacted.pdf.txt.json
98.96% in topic 14.	zero-carbon-bill/input/00066_Anonymous.pdf.txt.json
99.80% in topic 0.	zero-carbon-bill/input/00019_Linda_Bond_Redacted.pdf.txt.json
69.79% in topic 1.	zero-carbon-bill/input/01603_Raymond_Philip_Tremewan_Redacted.pdf.txt.json
99

In [ ]:
def plot_10_most_common_words(count_data, count_vectorizer):
    words = count_vectorizer.get_feature_names()
    total_counts = numpy.zeros(len(words))
    for t in count_data:
        total_counts += t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = numpy.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    seaborn.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    seaborn.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()

# Visualise the 10 most common words
plot_10_most_common_words(vectorized_data, vectorizer)


In [ ]:
# NOTE: This step takes a long while to return
prepared_data = pyLDAvis.sklearn.prepare(lda, vectorized_data, vectorizer)
prepared_data

In [ ]:
# WAIT here for previous step to return
